In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime as dt
from dateutil.parser import *
import random
import time
import json
from numpy import argmax
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score,classification_report, roc_auc_score,accuracy_score, \
precision_score, f1_score, recall_score, roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import re
import seaborn as sns
from imblearn.over_sampling import SMOTE 

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

%config InlineBackend.figure_format = 'retina'
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12)

In [15]:
df_regression = pd.read_csv('../data/merged_train_final.csv')

In [16]:
test_df_regression = pd.read_csv('../data/merged_test_final.csv')

In [17]:
x = df_regression.drop(['wnvpresent'],axis=1)
y = df_regression['wnvpresent']

y.value_counts(normalize=True)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [18]:
x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

In [19]:
sm = SMOTE(random_state=42)

x_train, y_train = sm.fit_resample(x_train,y_train)

In [20]:
ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

In [21]:
# Instantiate models
models = {'lr': LogisticRegression(max_iter=5_000, random_state=42, solver='saga'),
          'rf': RandomForestClassifier(random_state=42),
          'gb': GradientBoostingClassifier(random_state=42),
          'et': ExtraTreesClassifier(random_state=42),
          'ada': AdaBoostClassifier(random_state=42),
          'svc': SVC(random_state=42, probability=True),
}

In [30]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1)
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [31]:
# create list to store results
model_eval =[]

In [32]:
lr_base = get_model_scores('lr_base', 'lr')

METRICS-->


{'model_name': 'lr_base',
 'model': 'lr',
 'train_score': 0.9572568940493469,
 'test_score': 0.9360561056105611,
 'recall': 0.06451612903225806,
 'specificity': 0.9830434782608696,
 'precision': 0.1702127659574468,
 'train_auc': 0.9881001472443814,
 'test_auc': 0.7398650070126227,
 'is_tuned': False}

True Negatives: 2261
False Positives: 39
False Negatives: 116
True Positives: 8


Pipeline(steps=[('lr',
                 LogisticRegression(max_iter=5000, random_state=42,
                                    solver='saga'))])

In [28]:
lr = LogisticRegression()

lr_scores = cross_val_score(lr, x_train_sc , y_train, cv=5)

lr.fit(x_train_sc,y_train)

display(np.mean(lr_scores))

display(lr.score(x_train_sc,y_train))

display(lr.score(x_test_sc,y_test))

0.9569264955894103

0.9585867361714451

0.9306930693069307

In [39]:
predictions = lr.predict(x_test_sc)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
y_test_pred_prob = lr.predict_proba(x_test_sc)[:,1]
y_train_pred_prob = lr.predict_proba(x_train_sc)[:,1]

train_auc = roc_auc_score(y_train, y_train_pred_prob)
test_auc = roc_auc_score(y_test, y_test_pred_prob)

display(train_auc)
display(test_auc)

In [44]:
svm = SVC(probability=True)

svm_scores = cross_val_score(svm, x_train_sc , y_train, cv=5)

svm.fit(x_train_sc,y_train)

display(np.mean(svm_scores))

display(svm.score(x_train_sc,y_train))

display(svm.score(x_test_sc,y_test))

0.9556238522757143

0.9614827686070084

0.9352310231023102

In [45]:
predictions = svm.predict(x_test_sc)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
y_test_pred_prob = svm.predict_proba(x_test_sc)[:,1]
y_train_pred_prob = svm.predict_proba(x_train_sc)[:,1]

train_auc = roc_auc_score(y_train, y_train_pred_prob)
test_auc = roc_auc_score(y_test, y_test_pred_prob)

display(train_auc)
display(test_auc)

0.9937922123146546

0.7117353640230173